### Compare Subnational to SVI
Compare subnational data to SVI data in the USA.

#### 1. Clean Data

In [1]:
# Import stuff
import numpy as np
import pandas as pd
import os

In [2]:
# set wd
wd_path = "C:\\Users\\rcompos\\OneDrive - North Carolina State University\\Documents\\Research\\SVI_Flood_Project"
os.chdir(wd_path)
os.getcwd() # get wd
#os.listdir() # get available data

'c:\\Users\\rcompos\\OneDrive - North Carolina State University\\Documents\\Research\\SVI_Flood_Project'

In [3]:
# Set data paths
sub = pd.read_csv("Subnational\\sub_pov - selections.csv")
svi_cnty = pd.read_csv("SVI\\SVI2020_US_COUNTY.csv")
svi_tra = pd.read_csv("SVI\\SVI2020_US_tract.csv")

In [36]:
# View data
#sub.head(5)
#svi_cnty.head(5)
#svi_tra.head(5)
#sub.dtypes
#svi_cnty.dtypes


In [35]:
# Prep sub data to add to svi
sub_usa = sub[sub['code'] == 'USA'] # select usa data
#len(sub_usa) # 51, includes D.C.
#print(sub_usa[['STATE']].to_string(index=False)) # get list of states

sub_usa = sub_usa.rename(columns = {'State':'STATE','Mean Rank (SVI)':'sub_mean_rank'}) # rename column to match svi data
sub_usa = sub_usa[['STATE','sub_mean_rank']] # select columns
sub_usa['sub_mean_rank'] = sub_usa['sub_mean_rank'].astype(float)
#print(sub_usa.dtypes)

#sub_usa.head()


STATE             object
sub_mean_rank    float64
dtype: object


In [37]:
# Prep svi data by grouping by state and one attribute
#svi_st_m = svi_cnty.groupby('STATE')['E_POV150'].mean().reset_index(name='svi_pov')
#svi_st_m = svi_st_m[['STATE','svi_pov']] # select columns

51

In [87]:
# Prep svi data by grouping by state and multiple attributes
svi_st_m = svi_cnty.groupby('STATE').mean().reset_index()
svi_st_m = svi_st_m.iloc[:,np.r_[0, 11:40]]
#len(svi_st_m) # 51, includes D.C.
#svi_st_m.head(10)
#print(svi_st_m[['STATE']].to_string(index=False)) # get list of states

In [88]:
# Combine datasets
svi_merged = pd.merge(sub_usa, svi_st_m, on =['STATE'], how = 'left')
svi_merged.head()
#svi_merged.dtypes

,STATE,sub_mean_rank,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,...,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH
0,Alabama,109.611111,18423.119403,1246.850746,1870.791045,308.029851,6720.477612,524.014925,6551.447761,572.686567,...,212.074627,25388.641791,590.820896,2491.522388,248.134328,4421.358209,425.985075,469.358209,137.537313,1669.388060
1,Alaska,105.333333,4160.066667,406.200000,880.966667,136.466667,2061.600000,175.966667,1105.866667,157.700000,...,116.066667,9900.366667,3491.533333,808.433333,97.833333,472.033333,88.333333,550.900000,94.733333,787.566667
2,Arizona,102.777778,108363.600000,3027.400000,13290.533333,790.266667,48460.866667,1308.666667,39194.600000,1166.133333,...,979.000000,219356.133333,18233.733333,23704.666667,734.000000,20927.066667,846.600000,7766.866667,559.533333,10266.266667
3,Arkansas,107.166667,10643.733333,896.413333,956.053333,219.133333,3602.440000,373.240000,3471.466667,387.946667,...,166.946667,11423.653333,434.253333,1190.440000,169.280000,2145.173333,293.506667,433.306667,131.826667,958.240000
4,California,105.277778,139035.517241,3439.482759,21191.017241,945.948276,71735.517241,1533.637931,73905.827586,1530.775862,...,1505.603448,430704.793103,2924.913793,43272.913793,940.327586,8890.793103,526.068966,18547.206897,715.448276,15868.310345


#### 2. Compare w/ simple linear correlation

Notes - closer to 0 is less correlated & closer to 1 or -1 is more correlated, datasets must be same length

Source - https://www.geeksforgeeks.org/exploring-correlation-in-python/

In [91]:
# Find the pearson correlations matrix
svi_corr = svi_merged.corr(method = 'pearson') # run correlation
svi_corr[['sub_mean_rank']].sort_values(by='sub_mean_rank', ascending=False).head(10) # order results

,sub_mean_rank
sub_mean_rank,1.000000
M_MOBILE,0.276743
M_AGE17,0.205463
E_MOBILE,0.193927
M_UNINSUR,0.187677
M_NOHSDP,0.173680
M_POV150,0.152580
M_UNEMP,0.129511
E_UNINSUR,0.128021
E_POV150,0.124820
